### Import Libraries

In [1]:
import requests
import json
import pandas as pd
import yahoo_fin.stock_info as si
from datetime import datetime, timedelta

### Get AUD to USD Conversion

In [2]:
# Convert AUD to USD

def convert_currency_yahoofin(src, dst):
    # construct the currency pair symbol
    symbol = f"{src}{dst}=X"
    # extract minute data of the recent 2 days
    latest_data = si.get_data(symbol, interval="1m", start_date=datetime.now() - timedelta(days=2))
    # get the latest datetime
    last_updated_datetime = latest_data.index[-1].to_pydatetime()
    # get the latest price
    latest_price = latest_data.iloc[-1].close
    # return the latest datetime with the converted amount
    print(f"Yahoo Finance USD to AUD Conversione ($ {latest_price:.3f}) Price last updated {last_updated_datetime}")
    return  latest_price

### Get AUD -USD Conversion Rate
aud_convert = convert_currency_yahoofin("USD","AUD")

Yahoo Finance USD to AUD Conversione ($ 1.447) Price last updated 2022-07-21 05:39:07


### Initialise Functions

In [17]:
#function to extract data from btcmarkets
def crypto_price_btcmarkets(crypto_ticker,btcmarkets_api):

    # Gather prices
    crypto_bid = btcmarkets_api["bestBid"]
    crypto_ask = btcmarkets_api["bestAsk"]
    crypto_last = btcmarkets_api["lastPrice"]
    
    # print(f"{crypto_ticker} Buy: ${float(crypto_bid):,}")
    # print(f"{crypto_ticker} Sell: ${float(crypto_ask):,}")
    # print(f"{crypto_ticker} Last Transaction: ${float(crypto_last):,}")
    
    #return as dictionary
    return{"btcmarkets bid":crypto_bid,"btcmarkets ask":crypto_ask,"btcmarkets sell":crypto_last}

#function to extract data from coinspot
def crypto_price_coinspot(crypto_ticker):
    
    # Gather prices
    crypto_bid = coinspot_data["prices"][crypto_ticker]["bid"]
    crypto_ask = coinspot_data["prices"][crypto_ticker]["ask"]
    crypto_last = coinspot_data["prices"][crypto_ticker]["last"]
    
    # print(f"{crypto_ticker} Buy: ${float(crypto_bid):,}")
    # print(f"{crypto_ticker} Sell: ${float(crypto_ask):,}")
    # print(f"{crypto_ticker} Last Transaction: ${float(crypto_last):,}")

        #return as dictionary
    return{"coinspot bid":crypto_bid,"coinspot ask":crypto_ask,"coinspot sell":crypto_last}

### Execute GET request with URL

#function to extract data from coinspot
def get_api(coin_url):

    # Execute get request
    requests.get(coin_url)

    ### Store response as variable

    # Execute GET request and store response
    coin_response_data = requests.get(coin_url)

    ### Retrieve API output using `content` attribute

    # Get content
    coin_response_content = coin_response_data.content

    # print(response_content)

    # Formatting as json
    coin_data = coin_response_data.json()
    
    #Print data as formatted json with indents
    # print(json.dumps(coinspot_data, indent=4))
    
    return coin_data

#function to extract data from alternative
def crypto_price_alternative(id_,url_):
    
    # Execute get request
    requests.get(url_)

    # Execute GET request and store response
    response_data = requests.get(url_)

    # Get content
    response_content = response_data.content

    # Formatting as json
    data = response_data.json()

    crypto_price = data["data"][id_]["quotes"]["USD"]["price"]
    crypto_vol = data["data"][id_]["quotes"]["USD"]["volume_24h"]
   
    crypto_ask = crypto_price * aud_convert
    aud_vol = crypto_vol * aud_convert

    # print(f"{id_} is ${price:,.2f}")
    print(f"AUD volume is approximately ${aud_vol:,.2f}")

    return {"alternative ask":crypto_ask}

    #function to extract data from independent
def crypto_price_independent(url_):
    
    # Gather prices
    crypto_bid = independent_data_btc["CurrentLowestOfferPrice"]
    crypto_ask = independent_data_btc["CurrentHighestBidPrice"]
    crypto_last = independent_data_btc["LastPrice"]
    
    # print(f"{crypto_ticker} Buy: ${float(crypto_bid):,}")
    # print(f"{crypto_ticker} Sell: ${float(crypto_ask):,}")
    # print(f"{crypto_ticker} Last Transaction: ${float(crypto_last):,}")

        #return as dictionary
    return{"independent bid":crypto_bid,"independent ask":crypto_ask,"independent sell":crypto_last}

### Initialise API calls

In [23]:
# Create variable to hold request url
coinspot_url = "https://www.coinspot.com.au/pubapi/latest"

btcmarkets_url_btc ="https://api.btcmarkets.net/v3/markets/BTC-AUD/ticker"
btcmarkets_url_eth ="https://api.btcmarkets.net/v3/markets/ETH-AUD/ticker"

alternative_btc = "https://api.alternative.me/v2/ticker/Bitcoin/"
alternative_eth = "https://api.alternative.me/v2/ticker/Ethereum/"

independent_btc = "https://api.independentreserve.com/Public/GetMarketSummary?primaryCurrencyCode=xbt&secondaryCurrencyCode=aud"
independent_eth = "https://api.independentreserve.com/Public/GetMarketSummary?primaryCurrencyCode=eth&secondaryCurrencyCode=aud"

### GET API Data

In [24]:
# Return the json file using get_api function
coinspot_data = get_api(coinspot_url)

btcmarkets_data_btc = get_api(btcmarkets_url_btc)
btcmarkets_data_eth = get_api(btcmarkets_url_eth)

alternative_data_btc = get_api(alternative_btc)
alternative_data_eth  = get_api(alternative_eth)

independent_data_btc = get_api(independent_btc)
independent_data_eth = get_api(independent_eth)


### Convert to Dictionaries

In [25]:
# Run Coinspot Functions
btc_coinspot = crypto_price_coinspot(crypto_ticker = "btc")
eth_coinspot = crypto_price_coinspot(crypto_ticker = "eth")

#Run btcmarkets functions
btc_btcmarkets = crypto_price_btcmarkets(crypto_ticker = "btc",btcmarkets_api = btcmarkets_data_btc)
eth_btcmarkets = crypto_price_btcmarkets(crypto_ticker = "eth",btcmarkets_api = btcmarkets_data_eth)

#Run btcmarkets functions
btc_alternative = crypto_price_alternative(id_ = "1",url_ = alternative_btc)
eth_alternative  = crypto_price_alternative(id_ = "1027",url_  = alternative_eth )

#Run independent functions
btc_independent = crypto_price_independent(independent_btc)
eth_independent = crypto_price_independent(independent_eth)

#Join Dictionaries
btcmarkets_prices = {"btc": btc_btcmarkets,"eth" :eth_btcmarkets}
coinspot_prices = {"btc": btc_coinspot,"eth" :eth_coinspot}
alternative_prices = {"btc": btc_alternative,"eth" :eth_alternative}
independent_prices = {"btc": btc_independent,"eth" :eth_independent}

AUD volume is approximately $67,830,592,609.50
AUD volume is approximately $32,085,275,006.88


### Create DataFrame

In [26]:
# Convert Dictionaries to Dataframe
coinspot_df = pd.DataFrame.from_dict(coinspot_prices,orient = "index")
btcmarkets_df = pd.DataFrame.from_dict(btcmarkets_prices,orient = "index")
alternative_df = pd.DataFrame.from_dict(alternative_prices,orient = "index")
independent_df = pd.DataFrame.from_dict(independent_prices,orient = "index")


In [27]:
# Join dataframes
all_prices = pd.concat([coinspot_df,btcmarkets_df,independent_df,alternative_df], axis="columns", join="outer")

#Show Data
all_prices.head()

,coinspot bid,coinspot ask,coinspot sell,btcmarkets bid,btcmarkets ask,btcmarkets sell,independent bid,independent ask,independent sell,alternative ask
btc,33188,33308.99999995,33188.30000001,33165.73,33200,33236.99,33209.99,33156.11,33074.99,33132.334031
eth,2139.51000004,2164.47528457,2148.8,2158.58,2162.24,2160.14,33209.99,33156.11,33074.99,2152.552717


### Convert to float and perform basic calculations

In [28]:
#Show Data
all_prices = all_prices.astype(float).copy()

all_prices["bid_diff"] = all_prices["coinspot bid"] - all_prices["btcmarkets bid"]
all_prices["ask_diff"] = all_prices["coinspot ask"] - all_prices["btcmarkets ask"]
all_prices["sell_diff"] = all_prices["coinspot sell"] - all_prices["btcmarkets sell"]

all_prices.head()

,coinspot bid,coinspot ask,coinspot sell,btcmarkets bid,btcmarkets ask,btcmarkets sell,independent bid,independent ask,independent sell,alternative ask,bid_diff,ask_diff,sell_diff
btc,33188.00,33309.000000,33188.3,33165.73,33200.00,33236.99,33209.99,33156.11,33074.99,33132.334031,22.27,109.000000,-48.69
eth,2139.51,2164.475285,2148.8,2158.58,2162.24,2160.14,33209.99,33156.11,33074.99,2152.552717,-19.07,2.235285,-11.34
